In [ ]:
import torch
from torch import nn
import torch.optim as optim
import numpy as np
from torch.utils.data import TensorDataset,DataLoader

from dataset_slice import data2prepro,newprepro
from test_and_train import train_and_evaluate,train_one_epoch,evaluate,evaluate_model
from data_prepare import prepare_data,val_prepare_data
from visual_data import visualize_tsne,plot_model_confusion_matrix
from TFDAVIT import TFDAVIT
from torchinfo import summary

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = TFDAVIT(
        seq_len = 2048,
        patch_size = 16,
        num_classes = 10,
        windowsize=4,
        dim = 32,
        depth = 1,
        heads = 4,
        mlp_ratio = 4,
        dropout = 0.4,  
        channels=12,
    ).to(device)

summary(model, input_size=(1, 1, 2048))

Layer (type:depth-idx)                                  Output Shape              Param #
TFSCDAVIT                                               [1, 10]                   263
├─FFT_branch: 1-1                                       [1, 1, 2049]              --
│    └─LayerNorm: 2-1                                   [1, 1, 2049]              4,098
├─Conv1d: 1-2                                           [1, 32, 2048]             160
├─BatchNorm1d: 1-3                                      [1, 32, 2048]             64
├─Conv1d: 1-4                                           [1, 32, 256]              9,248
├─AdaptiveAvgPool1d: 1-5                                [1, 1, 1]                 --
├─Conv1d: 1-6                                           [1, 12, 1]                48
├─Sigmoid: 1-7                                          [1, 12, 1]                --
├─Sequential: 1-8                                       [1, 4, 4096]              --
│    └─Conv1d: 2-2                                  

In [ ]:
path = 'E:\\pythontask\\12kh\\0'
pretrain_X, pretrain_Y,preval_X,preval_Y,pretest_X, pretest_Y = newprepro(d_path=path,
                                                length=2048,
                                                number=400,
                                                strides=256,
                                                normal=False,
                                                valid=True,
                                                rate=[0.5, 0.125, 0.375],small=9,addnoise=False,snr=-8)

# path = 'E:\\pythontask\\Paderborndata\\'
# pretrain_X, pretrain_Y,preval_X,preval_Y,pretest_X, pretest_Y = data2prepro(d_path=path,
#                                                 length=4096,
#                                                 number=2300,
#                                                 strides=2048,
#                                                 normal=False,
#                                                 valid=True,
#                                                 rate=[0.5, 0.25, 0.25],small=0,addnoise=False,snr=-6)

In [37]:
# path = 'E:\\pythontask\\12kh\\0'
# pretrain_X0, pretrain_Y0,preval_X0,preval_Y0,pretest_X0, pretest_Y0 = newprepro(d_path=path,
#                                                 length=1024,
#                                                 number=100,
#                                                 strides=1024,
#                                                 normal=False,
#                                                 valid=True,
#                                                 rate=[0.5, 0.25, 0.25],small=0,addnoise=True,snr=-8)

# path = 'E:\\pythontask\\12kh\\1'
# pretrain_X1, pretrain_Y1,preval_X1,preval_Y1,pretest_X1, pretest_Y1 = newprepro(d_path=path,
#                                                 length=1024,
#                                                 number=100,
#                                                 strides=1024,
#                                                 normal=False,
#                                                 valid=True,
#                                                 rate=[0.5, 0.25, 0.25],small=0,addnoise=True,snr=-8)

# path = 'E:\\pythontask\\12kh\\2'
# pretrain_X2, pretrain_Y2,preval_X2,preval_Y2,pretest_X2, pretest_Y2 = newprepro(d_path=path,
#                                                 length=1024,
#                                                 number=100,
#                                                 strides=1024,
#                                                 normal=False,
#                                                 valid=True,
#                                                 rate=[0.5, 0.25, 0.25],small=0,addnoise=True,snr=-8)

# path = 'E:\\pythontask\\12kh\\3'
# pretrain_X3, pretrain_Y3,preval_X3,preval_Y3,pretest_X3, pretest_Y3 = newprepro(d_path=path,
#                                                 length=1024,
#                                                 number=100,
#                                                 strides=1024,
#                                                 normal=False,
#                                                 valid=True,
#                                                 rate=[0.5, 0.25, 0.25],small=0,addnoise=True,snr=-8)

# pretrain_X = np.concatenate([pretrain_X0,pretrain_X1,pretrain_X2,pretrain_X3],axis=0)
# pretrain_Y = np.concatenate([pretrain_Y0,pretrain_Y1,pretrain_Y2,pretrain_Y3],axis=0)
# preval_X = np.concatenate([preval_X0,preval_X1,preval_X2,preval_X3],axis=0)
# preval_Y = np.concatenate([preval_Y0,preval_Y1,preval_Y2,preval_Y3],axis=0)
# pretest_X = np.concatenate([pretest_X0,pretest_X1,pretest_X2,pretest_X3],axis=0)
# pretest_Y = np.concatenate([pretest_Y0,pretest_Y1,pretest_Y2,pretest_Y3],axis=0)

In [ ]:
pretrain_X, pretest_X, pretrain_Y, pretest_Y = prepare_data(pretrain_X, pretest_X, pretrain_Y, pretest_Y, device,length = 2048)
preval_X,preval_Y= val_prepare_data(preval_X,preval_Y,device,length=2048)

In [ ]:
batch_size = 64
num_epochs = 50

pretrain_dataset = TensorDataset(pretrain_X, pretrain_Y)
pretest_dataset = TensorDataset(pretest_X, pretest_Y)
preval_dataset = TensorDataset(preval_X, preval_Y)
pretrain_loader = DataLoader(pretrain_dataset, batch_size=batch_size, shuffle=True)
pretest_loader = DataLoader(pretest_dataset, batch_size=batch_size, shuffle=False)
preval_loader = DataLoader(preval_dataset, batch_size=batch_size, shuffle=False)

preoptimizer = optim.Adam(model.parameters(), lr=0.0005, weight_decay=1e-3)
precriterion = nn.CrossEntropyLoss()
scheduler = optim.lr_scheduler.StepLR(preoptimizer, step_size=10, gamma=1)

train_and_evaluate(
    model=model,
    train_loader=pretrain_loader,
    test_loader=preval_loader,
    criterion=precriterion,
    optimizer=preoptimizer,
    scheduler=scheduler,
    num_epochs=num_epochs,
    train_one_epoch=train_one_epoch,
    evaluate=evaluate
)

In [ ]:
accuracy = evaluate_model(model, pretest_loader, device)

In [ ]:
# plot_model_confusion_matrix(model, pretest_loader, device, class_names=['0', '1', '2','3', '4','5', '6', '7','8', '9'])

In [ ]:
# visualize_tsne(model, pretest_loader, device)